## This notebook is a work in progress.
See the corresponding Python notebook for explanation of why we need each step.

In [69]:
## store api-key globally
apikey = '0fa36e42-a7cf-43eb-9511-7729243e10a2'  

In [70]:
## load neededd libraries
library(httr)
library(xml2)

In [76]:
## post to the CAS endpoint
response <- POST('https://utslogin.nlm.nih.gov/cas/v1/api-key', encode='form', body=list(apikey = apikey))
## print out the status_code and content_type
status_code(response)
headers(response)$`content-type`
## store globally so it is available to the next cell
globalVariables.response = response    ## must store globally

[1] 201

[1] "text/html;charset=UTF-8"

### content is already XML
In R, the content of the response is already parsed as an XML/HTML document, so we need only find the form's action attribute using an XPath expression.

In [77]:
action_uri = xml_text(xml_find_first(content(response), '//form/@action'))
action_uri

[1] "https://utslogin.nlm.nih.gov/cas/v1/api-key/TGT-1939862-kccb0GABPeAS6aANBtRlQlSP442XjL1QadLueW2PyES5epUQa9-cas"

In [78]:
response <- POST(action_uri, encode='form', body=list(service = 'http://umlsks.nlm.nih.gov'))
## R tries to automatically guess the body content type, let's tell it this is pure text
ticket = content(response, 'text')
ticket

[1] "ST-16041988-njtbsoa70ubAYpvgJC4d-cas"

In [79]:
version = 'current'
query_string = 'diabetic foot'
search_uri = paste('https://uts-ws.nlm.nih.gov/rest/search/', version, sep='')
response <- GET(search_uri, query=list(ticket=ticket, string=query_string, searchType='exact'))

## print out some of the results
search_uri
status_code(response)
headers(response)$`content-type`

[1] "https://uts-ws.nlm.nih.gov/rest/search/current"

[1] 200

[1] "application/json;charset=UTF-8"

In [80]:
content(response)

$pageSize
[1] 25

$pageNumber
[1] 1

$result
$result$classType
[1] "searchResults"

$result$results
$result$results[[1]]
$result$results[[1]]$ui
[1] "C0206172"

$result$results[[1]]$rootSource
[1] "MTH"

$result$results[[1]]$uri
[1] "https://uts-ws.nlm.nih.gov/rest/content/2017AB/CUI/C0206172"

$result$results[[1]]$name
[1] "Diabetic Foot"